How would one actually get sentiment? Well, that's something that can be learned from a corpus of words in much the same way as features were extracted from images. This process is called embedding, with the idea being that words and associated words are clustered as vectors in a multi-dimensional space.

![Word embeding](https://miro.medium.com/max/1502/0*C7uYogpD7l7YPSVs)

[Google News and Leo Tolstoy: Visualizing Word2Vec Word Embeddings using t-SNE](https://towardsdatascience.com/google-news-and-leo-tolstoy-visualizing-word2vec-word-embeddings-with-t-sne-11558d8bd4d)

## 1. Loading the IMBD dataset

 There's a library called TensorFlow Data Services or TFTS for short, and that contains many data sets and lots of different categories

In [1]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed

# tf.enable_eager_execution()
# !pip install -q tensorflow-datasets

2.0.0-beta1


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /Users/hkmac/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [5]:
import numpy as np
# split data into trian and test
train_data, test_data = imdb['train'], imdb['test']
imdb['train']

<_OptionsDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [7]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []
# iterate over trianing data, extract the sentences and the label
# the value of s and l are tensors, calling there numpy method can extract their value
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
# turn the list of labels into numpy arrays    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

## 2. Tokenize our sentences

In [8]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [22]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b oh yeah jenna jameson did it again yeah baby this movie rocks it was one of the 1st movies i saw of her and i have to say i feel in love with her she was great in this move br br her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing br br i hope this comment helps and u can buy the movie the storyline is awesome is very unique and i'm sure u are going to like it jenna amazed us once more and no wonder the movie won so many
b"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the

## 3. Define and train our model

In a movie review, it might say that the movie was dull and boring, or it might say that it was fun and exciting. So what if you could pick a `vector` in a higher-dimensional space say 16 dimensions, and words that are found together are given similar vectors. Then over time, words can begin to `cluster` together. The meaning of the words can come from the labeling of the dataset. So in this case, we say a negative review and the words dull and boring show up a lot in the negative review so that they have `similar sentiments`, and they are close to each other in the sentence. Thus their vectors will be similar.

As the neural network trains, it can then learn these vectors associating them with the labels to come up with what's called an embedding i.e., the vectors for each word with their associated sentiment. The results of the embedding will be a 2D array with the length of the sentence and the embedding dimension for example 16 as its size. So we need to flatten it out in much the same way as we needed to flatten out our images. We then feed that into a dense neural network to do the classification. 

In [1]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

NameError: name 'tf' is not defined

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


The output of the first layer: 2-D array. 120 is the length of sentence, and 16 is the embedding dimension. We flatten it to 120X16 = 1920. Often in natural language processing, a different layer type than a flatten is used, and this is a `global average pooling 1D`. The reason for this is the size of the output vector being fed into the dense. 

In [16]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 2s 65us/sample - loss: 0.1911 - accuracy: 0.9302 - val_loss: 0.3873 - val_accuracy: 0.8429
Epoch 2/10
25000/25000 [==============================] - 2s 62us/sample - loss: 0.1729 - accuracy: 0.9378 - val_loss: 0.4193 - val_accuracy: 0.8368
Epoch 3/10
25000/25000 [==============================] - 2s 63us/sample - loss: 0.1585 - accuracy: 0.9450 - val_loss: 0.4478 - val_accuracy: 0.8358
Epoch 4/10
25000/25000 [==============================] - 2s 63us/sample - loss: 0.1444 - accuracy: 0.9506 - val_loss: 0.4830 - val_accuracy: 0.8279
Epoch 5/10
25000/25000 [==============================] - 2s 63us/sample - loss: 0.1322 - accuracy: 0.9555 - val_loss: 0.5224 - val_accuracy: 0.8249
Epoch 6/10
25000/25000 [==============================] - 2s 63us/sample - loss: 0.1223 - accuracy: 0.9619 - val_loss: 0.5570 - val_accuracy: 0.8202
Epoch 7/10
25000/25000 [==============================] 

## 4. Visualize embeding

In [19]:
# getting the result of embeding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
# we have 10000 words 

(10000, 16)


In [20]:
# a helper func to reverse our word index, now the key is the word, and the value is the token for the word
# look through the padded list to decode the token back into the words.
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [21]:
# write the vectors and their metadata auto files.
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    # metedata array: write out the words
    out_m.write(word + "\n")
    # vector array: write each of the items in the array of embeddings.
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

Visualize using: https://projector.tensorflow.org/